In [1]:
import autograd.numpy as np
import autograd.numpy.random as npr
from neuralfingerprint import load_data
from autograd import grad
from keras.layers import Embedding, Dense

Using TensorFlow backend.


In [2]:
### Getting HCEP data into SMILES format
task_params = {'target_name' : 'measured log solubility in mols per litre',
               'data_file'   : 'delaney.csv'}
N_train = 672
N_val   = 0
N_test  = 168

smiles_limit = 70

model_params = dict(fp_length=50,    # Usually neural fps need far fewer dimensions than morgan.
                    fp_depth=4,      # The depth of the network equals the fingerprint radius.
                    conv_width=20,   # Only the neural fps need this parameter.
                    h1_size=100,     # Size of hidden layer of network on top of fps.
                    L2_reg=np.exp(-2))

print "Loading data..."
traindata, valdata, testdata = load_data(
    task_params['data_file'], (N_train, N_val, N_test),
    input_name='smiles', target_name=task_params['target_name'])
train_inputs, train_targets = traindata
val_inputs,   val_targets   = valdata
test_inputs,  test_targets  = testdata

print "Task params", task_params

from rdkit import Chem

SMILES_CHARS = [' ',
                  '#', '%', '(', ')', '+', '-', '.', '/',
                  '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                  '=', '@',
                  'A', 'B', 'C', 'F', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P',
                  'R', 'S', 'T', 'V', 'X', 'Z',
                  '[', '\\', ']',
                  'a', 'b', 'c', 'e', 'g', 'i', 'l', 'n', 'o', 'p', 'r', 's',
                  't', 'u']
smi2index = dict( (c,i) for i,c in enumerate( SMILES_CHARS ) )
index2smi = dict( (i,c) for i,c in enumerate( SMILES_CHARS ) )

no_input = N_train
X_smiles = train_inputs[0:no_input]
no_test = N_test
X_smiles_test = test_inputs[0:no_test]

def smiles_encoder_2( smiles, maxlen=120 ):
    smiles = Chem.MolToSmiles(Chem.MolFromSmiles( smiles ))
    X = np.zeros( ( 1, maxlen ) )
    for i, c in enumerate( smiles ):
        X[:,i] = smi2index[c]
    return X
 
def smiles_decoder_2( X ):
    smi = ''
    for i in X:
        smi += index2smi[ i ]
    return smi

Loading data...
Task params {'target_name': 'measured log solubility in mols per litre', 'data_file': 'delaney.csv'}


In [3]:
# Training data

mat_2 = np.zeros([len(X_smiles),120])
mat_3 = np.zeros([len(X_smiles),smiles_limit])
for i in range(0,len(X_smiles)): 
    mat_2[i,:] = smiles_encoder_2(X_smiles[i])
   
for i in range(0,len(mat_2)):
    mat_3[i] = mat_2[i][0:smiles_limit]
    
# Test data
# X = smiles_encoder_2(X_smiles[0])
mat_4 = np.zeros([len(X_smiles_test),120])
mat_5 = np.zeros([len(X_smiles_test),smiles_limit])
for i in range(0,len(X_smiles_test)): 
    mat_4[i,:] = smiles_encoder_2(X_smiles_test[i])
   
for i in range(0,len(mat_4)):
    mat_5[i] = mat_4[i][0:smiles_limit]

In [4]:
# RNN / GRU - MODEL
import edward as ed
import numpy as np
import tensorflow as tf
from keras.layers import Embedding, Dense
from edward.models import *
from edward.util import Progbar

H = 4
D = 2
V = 56
E = 2
batch_size = N_test
M = 10
nb_steps = 70

N=len(mat_3)
X_train = mat_3

# Training labels
y_train = train_targets[0:no_input]
y_train = np.reshape(y_train,(len(y_train),1))

# Test labels
y_test = test_targets[0:no_test]
y_test = np.reshape(y_test,(len(y_test),1))

In [5]:
with tf.variable_scope('model', reuse=True):

    # Weights in GRU
    Wfo = Normal(loc=tf.zeros([D, H]), scale=tf.ones([D, H]))
    Wro = Normal(loc=tf.zeros([H, H]), scale=tf.ones([H, H]))

    Wff = Normal(loc=tf.zeros([D, H]), scale=tf.ones([D, H]))
    Wrf = Normal(loc=tf.zeros([H, H]), scale=tf.ones([H, H]))

    Wfy = Normal(loc=tf.zeros([D, H]), scale=tf.ones([D, H]))
    Wry = Normal(loc=tf.zeros([H, H]), scale=tf.ones([H, H]))

    qWfo = Normal(loc=tf.Variable(tf.random_normal([D, H])), scale=tf.nn.softplus(tf.Variable(tf.random_normal([D, H]))))
    qWro = Normal(loc=tf.Variable(tf.random_normal([H, H])), scale=tf.nn.softplus(tf.Variable(tf.random_normal([H, H]))))

    qWff = Normal(loc=tf.Variable(tf.random_normal([D, H])), scale=tf.nn.softplus(tf.Variable(tf.random_normal([D, H]))))
    qWrf = Normal(loc=tf.Variable(tf.random_normal([H, H])), scale=tf.nn.softplus(tf.Variable(tf.random_normal([H, H]))))

    qWfy = Normal(loc=tf.Variable(tf.random_normal([D, H])), scale=tf.nn.softplus(tf.Variable(tf.random_normal([D, H]))))
    qWry = Normal(loc=tf.Variable(tf.random_normal([H, H])), scale=tf.nn.softplus(tf.Variable(tf.random_normal([H, H]))))
    
    # Placeholders 
    y_ph = tf.placeholder(tf.float32, [batch_size, 1], name='y_ph')

    x = tf.placeholder(tf.int32, [batch_size, nb_steps ], name='x')     
    
    # GRU cell
    def gru_cell(hprev, xt):
        #  output gate
        #import pdb; pdb.set_trace()
        i_o = tf.sigmoid(tf.matmul(xt,Wfo) + tf.matmul(hprev,Wro))
        #  forget gate
        i_f = tf.sigmoid(tf.matmul(xt,Wff) + tf.matmul(hprev,Wrf))
        #  intermediate
        y = tf.tanh(tf.matmul(xt,Wfy) + tf.matmul((i_f*hprev),Wry))
        # new state
        return (1-i_o)*y + (i_o*hprev)
      
    # Embedding (?)
    x_ = Embedding(V, D, name='Embedding')(x)
    
    # Initialise hidden state
    h = tf.zeros(shape=(batch_size, H)) # initial state

    for t in range(nb_steps-1):
        h = gru_cell(h, x_[:,t,:])
        
    # Variational Inference
    W1 = Normal(loc=tf.zeros([D, 1]), scale=tf.ones([D, 1]))
    W2 = Normal(loc=tf.zeros([H, D]), scale=tf.ones([H, D]))
    
    qW1 = Normal(loc=tf.Variable(tf.random_normal([D, 1])), scale=tf.nn.softplus(tf.Variable(tf.random_normal([D, 1]))))
    qW2 = Normal(loc=tf.Variable(tf.random_normal([H, D])), scale=tf.nn.softplus(tf.Variable(tf.random_normal([H, D]))))
    
    def fhw(h_in):
        fhw = tf.matmul(tf.sigmoid(tf.matmul(h_in, W2)), W1)    
        return fhw
      
    y = Normal(loc=fhw(h), scale=0.1 * tf.ones([batch_size,1]))

In [6]:
# INFERENCE
inference = ed.BB_alpha({W1: qW1, W2: qW2, 
                     Wfo: qWfo, Wro: qWro, 
                     Wff: qWff, Wrf: qWrf, 
                     Wfy: qWfy, Wry: qWry}, data={y: y_ph})

optimizer = tf.train.RMSPropOptimizer(0.001, epsilon=1.0)
inference.initialize(optimizer=optimizer,scale={y: len(X_train) / batch_size},alpha=0.5,logdir='log/n_samples_3') # always redefine inference before
sess = ed.get_session()
tf.global_variables_initializer().run()

n_epoch = 500
n_iter_per_epoch = 1000
avg_loss_array = np.zeros(n_epoch)

for epoch in range(n_epoch):
  avg_loss = 0.0

  pbar = Progbar(n_iter_per_epoch)
  for t in range(1, n_iter_per_epoch + 1):
    pbar.update(t)   
    batch = np.random.randint(0, len(X_train)-1, batch_size)
    info_dict = inference.update({x: X_train[batch], y_ph: y_train[batch]})
    avg_loss  += info_dict['loss']
    avg_loss_array[epoch] = avg_loss
    
  # Print a lower bound to the average marginal likelihood for an
  # image.
  avg_loss = avg_loss / n_iter_per_epoch
  avg_loss = avg_loss / batch_size
  print("log p(x) >= {:0.3f}".format(avg_loss))
  print("Epoch no : ", epoch)

1000/1000 [100%] ██████████████████████████████ Elapsed: 11s
log p(x) >= 2714.512
('Epoch no : ', 0)
1000/1000 [100%] ██████████████████████████████ Elapsed: 10s
log p(x) >= 1732.926
('Epoch no : ', 1)
1000/1000 [100%] ██████████████████████████████ Elapsed: 10s
log p(x) >= 1207.992
('Epoch no : ', 2)
1000/1000 [100%] ██████████████████████████████ Elapsed: 10s
log p(x) >= 1021.422
('Epoch no : ', 3)
1000/1000 [100%] ██████████████████████████████ Elapsed: 10s
log p(x) >= 954.985
('Epoch no : ', 4)
1000/1000 [100%] ██████████████████████████████ Elapsed: 9s
log p(x) >= 909.270
('Epoch no : ', 5)
1000/1000 [100%] ██████████████████████████████ Elapsed: 9s
log p(x) >= 879.043
('Epoch no : ', 6)
1000/1000 [100%] ██████████████████████████████ Elapsed: 10s
log p(x) >= 874.483
('Epoch no : ', 7)
1000/1000 [100%] ██████████████████████████████ Elapsed: 11s
log p(x) >= 866.318
('Epoch no : ', 8)
1000/1000 [100%] ██████████████████████████████ Elapsed: 10s
log p(x) >= 861.320
('Epoch no : ', 9

1000/1000 [100%] ██████████████████████████████ Elapsed: 9s
log p(x) >= 89.924
('Epoch no : ', 82)
1000/1000 [100%] ██████████████████████████████ Elapsed: 10s
log p(x) >= 89.957
('Epoch no : ', 83)
1000/1000 [100%] ██████████████████████████████ Elapsed: 9s
log p(x) >= 89.630
('Epoch no : ', 84)
1000/1000 [100%] ██████████████████████████████ Elapsed: 9s
log p(x) >= 89.330
('Epoch no : ', 85)
1000/1000 [100%] ██████████████████████████████ Elapsed: 9s
log p(x) >= 88.898
('Epoch no : ', 86)
1000/1000 [100%] ██████████████████████████████ Elapsed: 9s
log p(x) >= 89.849
('Epoch no : ', 87)
1000/1000 [100%] ██████████████████████████████ Elapsed: 9s
log p(x) >= 89.422
('Epoch no : ', 88)
1000/1000 [100%] ██████████████████████████████ Elapsed: 9s
log p(x) >= 88.810
('Epoch no : ', 89)
1000/1000 [100%] ██████████████████████████████ Elapsed: 9s
log p(x) >= 88.147
('Epoch no : ', 90)
1000/1000 [100%] ██████████████████████████████ Elapsed: 10s
log p(x) >= 88.903
('Epoch no : ', 91)
1000/100

1000/1000 [100%] ██████████████████████████████ Elapsed: 11s
log p(x) >= 84.455
('Epoch no : ', 163)
1000/1000 [100%] ██████████████████████████████ Elapsed: 11s
log p(x) >= 84.112
('Epoch no : ', 164)
1000/1000 [100%] ██████████████████████████████ Elapsed: 11s
log p(x) >= 86.097
('Epoch no : ', 165)
1000/1000 [100%] ██████████████████████████████ Elapsed: 10s
log p(x) >= 85.137
('Epoch no : ', 166)
1000/1000 [100%] ██████████████████████████████ Elapsed: 10s
log p(x) >= 83.784
('Epoch no : ', 167)
1000/1000 [100%] ██████████████████████████████ Elapsed: 10s
log p(x) >= 84.454
('Epoch no : ', 168)
1000/1000 [100%] ██████████████████████████████ Elapsed: 10s
log p(x) >= 83.616
('Epoch no : ', 169)
1000/1000 [100%] ██████████████████████████████ Elapsed: 9s
log p(x) >= 87.626
('Epoch no : ', 170)
1000/1000 [100%] ██████████████████████████████ Elapsed: 11s
log p(x) >= 86.946
('Epoch no : ', 171)
1000/1000 [100%] ██████████████████████████████ Elapsed: 10s
log p(x) >= 87.001
('Epoch no :

1000/1000 [100%] ██████████████████████████████ Elapsed: 10s
log p(x) >= 114.805
('Epoch no : ', 244)
1000/1000 [100%] ██████████████████████████████ Elapsed: 9s
log p(x) >= 95.806
('Epoch no : ', 245)
1000/1000 [100%] ██████████████████████████████ Elapsed: 10s
log p(x) >= 86.767
('Epoch no : ', 246)
1000/1000 [100%] ██████████████████████████████ Elapsed: 10s
log p(x) >= 88.280
('Epoch no : ', 247)
1000/1000 [100%] ██████████████████████████████ Elapsed: 10s
log p(x) >= 84.806
('Epoch no : ', 248)
1000/1000 [100%] ██████████████████████████████ Elapsed: 9s
log p(x) >= 89.146
('Epoch no : ', 249)
1000/1000 [100%] ██████████████████████████████ Elapsed: 9s
log p(x) >= 87.337
('Epoch no : ', 250)
1000/1000 [100%] ██████████████████████████████ Elapsed: 10s
log p(x) >= 83.422
('Epoch no : ', 251)
1000/1000 [100%] ██████████████████████████████ Elapsed: 10s
log p(x) >= 84.394
('Epoch no : ', 252)
1000/1000 [100%] ██████████████████████████████ Elapsed: 10s
log p(x) >= 84.265
('Epoch no : 

1000/1000 [100%] ██████████████████████████████ Elapsed: 10s
log p(x) >= 83.658
('Epoch no : ', 325)
1000/1000 [100%] ██████████████████████████████ Elapsed: 10s
log p(x) >= 83.320
('Epoch no : ', 326)
1000/1000 [100%] ██████████████████████████████ Elapsed: 10s
log p(x) >= 82.765
('Epoch no : ', 327)
1000/1000 [100%] ██████████████████████████████ Elapsed: 11s
log p(x) >= 84.210
('Epoch no : ', 328)
1000/1000 [100%] ██████████████████████████████ Elapsed: 11s
log p(x) >= 82.232
('Epoch no : ', 329)
1000/1000 [100%] ██████████████████████████████ Elapsed: 11s
log p(x) >= 82.892
('Epoch no : ', 330)
1000/1000 [100%] ██████████████████████████████ Elapsed: 10s
log p(x) >= 83.558
('Epoch no : ', 331)
1000/1000 [100%] ██████████████████████████████ Elapsed: 10s
log p(x) >= 82.481
('Epoch no : ', 332)
1000/1000 [100%] ██████████████████████████████ Elapsed: 10s
log p(x) >= 83.402
('Epoch no : ', 333)
1000/1000 [100%] ██████████████████████████████ Elapsed: 10s
log p(x) >= 82.141
('Epoch no 

1000/1000 [100%] ██████████████████████████████ Elapsed: 11s
log p(x) >= 83.148
('Epoch no : ', 406)
1000/1000 [100%] ██████████████████████████████ Elapsed: 11s
log p(x) >= 83.032
('Epoch no : ', 407)
1000/1000 [100%] ██████████████████████████████ Elapsed: 10s
log p(x) >= 88.231
('Epoch no : ', 408)
1000/1000 [100%] ██████████████████████████████ Elapsed: 10s
log p(x) >= 89.561
('Epoch no : ', 409)
1000/1000 [100%] ██████████████████████████████ Elapsed: 10s
log p(x) >= 85.316
('Epoch no : ', 410)
1000/1000 [100%] ██████████████████████████████ Elapsed: 13s
log p(x) >= 84.951
('Epoch no : ', 411)
1000/1000 [100%] ██████████████████████████████ Elapsed: 9s
log p(x) >= 84.496
('Epoch no : ', 412)
1000/1000 [100%] ██████████████████████████████ Elapsed: 10s
log p(x) >= 84.559
('Epoch no : ', 413)
1000/1000 [100%] ██████████████████████████████ Elapsed: 13s
log p(x) >= 83.381
('Epoch no : ', 414)
1000/1000 [100%] ██████████████████████████████ Elapsed: 12s
log p(x) >= 82.734
('Epoch no :

1000/1000 [100%] ██████████████████████████████ Elapsed: 10s
log p(x) >= 82.776
('Epoch no : ', 487)
1000/1000 [100%] ██████████████████████████████ Elapsed: 10s
log p(x) >= 83.479
('Epoch no : ', 488)
1000/1000 [100%] ██████████████████████████████ Elapsed: 10s
log p(x) >= 81.763
('Epoch no : ', 489)
1000/1000 [100%] ██████████████████████████████ Elapsed: 10s
log p(x) >= 82.020
('Epoch no : ', 490)
1000/1000 [100%] ██████████████████████████████ Elapsed: 10s
log p(x) >= 83.467
('Epoch no : ', 491)
1000/1000 [100%] ██████████████████████████████ Elapsed: 10s
log p(x) >= 81.648
('Epoch no : ', 492)
1000/1000 [100%] ██████████████████████████████ Elapsed: 10s
log p(x) >= 83.383
('Epoch no : ', 493)
1000/1000 [100%] ██████████████████████████████ Elapsed: 10s
log p(x) >= 85.390
('Epoch no : ', 494)
1000/1000 [100%] ██████████████████████████████ Elapsed: 11s
log p(x) >= 83.683
('Epoch no : ', 495)
1000/1000 [100%] ██████████████████████████████ Elapsed: 12s
log p(x) >= 83.645
('Epoch no 

In [7]:
# CRITICISM

X_test = mat_5

# Test labels
y_test = test_targets[0:no_test]
y_test = np.reshape(y_test,(len(y_test),1))

no_samples = 100
rmse_mat = np.zeros([no_samples])

for i in range(0,no_samples):
    test1 = sess.run({W1: qW1.sample(), W2: qW2.sample(),
                     Wfo: qWfo.sample(), Wro: qWro.sample(),
                     Wff: qWff.sample(), Wrf: qWrf.sample(),
                     Wfy: qWfy.sample(), Wry: qWry.sample()},{x: X_test})

    y_post = ed.copy(y, {W1: qW1, W2: qW2,
                         Wfo: qWfo, Wro: qWro,
                         Wff: qWff, Wrf: qWrf,
                         Wfy: qWfy, Wry: qWry})
    y_out = sess.run(y_post, feed_dict={x:X_test})
    rmse = np.sqrt(np.mean(np.square(y_out-y_test)))
    rmse_mat[i] = rmse
    
print('\n'.join(map(str, rmse_mat)))
print('y_out : ', y_out[0:20])
print('y_test : ', y_test[0:20])

0.9929041593721571
1.012463471552974
1.0179105645320639
1.0123429586328847
1.0119145182606966
1.013778846954851
1.0212178172111048
1.0174283263215507
1.001784922849191
1.0021098056878037
1.006992327058471
1.008227037311795
1.0139351176704874
1.0155020973529385
1.0044751634700142
1.0194850432496987
1.0073096783027486
1.011146492064049
1.0318000890776442
1.016485435846335
1.019748832480637
1.0108201301608513
1.009355269920094
1.019567072441417
1.0077974275015131
1.0243344171774744
1.0003564875905395
1.005574345350196
1.0135300160000251
1.0126736449086
1.0128685991816078
1.0175850724079294
1.0106048199637645
1.0257141528842226
0.99487637601603
1.017812409448729
1.0026411847365828
1.016298093026732
1.0087397760376766
0.999695132488453
1.011936254398499
1.0095477974803062
0.9954887832522257
1.0113915908382267
1.0276632400342904
1.0017935827445035
1.011560536546331
1.0184881481194301
1.0255769409625617
1.0033762447307155
1.0272012122438021
1.009789416347492
1.0063146701594636
0.9990913809823

In [8]:
print('\n'.join(map(str,avg_loss_array/(n_iter_per_epoch*batch_size))))

2714.51184593564
1732.9259829334078
1207.991830078125
1021.4223811383929
954.9854341052827
909.2698238002232
879.0425879836309
874.4833229166667
866.3182642764137
861.320048967634
855.1986450427827
858.2666569475447
841.4545100446428
801.1978603050595
702.043484468006
636.628015531994
587.0031179315476
542.5485011393229
511.58431591796875
480.49763364955356
446.257776390439
421.5240459914435
380.6511342308408
311.3348297061012
281.6835412248884
251.36911962890625
229.4030627092634
210.86434170386906
194.82230428059896
181.5800220889137
172.56294471958705
162.16378232538133
154.81340798804874
146.55701275344123
142.3529193231492
135.54391766066777
131.59633819870723
126.27113627115885
120.88568347167968
116.5631920282273
112.82978048851376
110.61196849423364
109.94137446521577
108.21654080054874
108.52465227399554
107.16754220726376
106.17273088146392
104.84063001650856
104.67253826032366
102.79303128487723
102.37146969749814
100.61287363978795
100.0470660749163
98.5722577718099
98.0709